# This is work in progress and not completed

In [199]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from itertools import combinations, product
from recsys.utils.utils import IDConverter

In [214]:
class BetslipDataset(Dataset):
    def __init__(self, user_df, event_df, bets_df, market_df, max_selections=5):
        self.max_selections = max_selections
        
        # Create ID mappings first
        self.user_id_to_idx = {uid: idx for idx, uid in enumerate(user_df['player_id'].unique())}
        self.event_id_to_idx = {eid: idx for idx, eid in enumerate(event_df['event_id'].unique())}
        self.market_id_to_idx = {mid: idx for idx, mid in enumerate(market_df['market_id'].unique())}
        
        # Filter bets to include only valid IDs
        valid_bets_df = bets_df[
            (bets_df['player_id'].isin(self.user_id_to_idx.keys())) &
            (bets_df['event_id'].isin(self.event_id_to_idx.keys())) &
            (bets_df['market_id'].isin(self.market_id_to_idx.keys()))
        ]
        
        print(f"Original bets: {len(bets_df)}, Valid bets: {len(valid_bets_df)}")
        
        # Process features
        self.user_features = self.process_user_features(user_df, valid_bets_df)
        self.event_features = self.process_event_features(event_df)
        self.market_features = self.process_market_features(market_df)
        
        # Create betslips from valid bets
        self.betslips = self.create_betslips(valid_bets_df)
        
        print(f"Processed {len(self.betslips)} valid betslips")
        print(f"User features shape: {self.user_features.shape}")
        print(f"Event features shape: {self.event_features.shape}")
        print(f"Market features shape: {self.market_features.shape}")

    def process_user_features(self, users_df, bets_df):
        # Calculate user betting patterns
        user_stats = bets_df.groupby('player_id').agg({
            'amount': ['mean', 'count'],
            'bet_odds': 'mean',
            'status': lambda x: (x == 'won').mean()
        }).reset_index()
        
        user_stats.columns = ['player_id', 'avg_stake', 'bet_count', 'avg_odds', 'win_rate']
        
        # Create feature matrix for all users in mapping
        feature_matrix = np.zeros((len(self.user_id_to_idx), 4))  # 4 features
        
        for _, row in user_stats.iterrows():
            if row['player_id'] in self.user_id_to_idx:
                idx = self.user_id_to_idx[row['player_id']]
                feature_matrix[idx] = [
                    row['avg_stake'], row['bet_count'],
                    row['avg_odds'], row['win_rate']
                ]
        
        return feature_matrix

    def process_event_features(self, events_df):
        # Calculate temporal features
        events_df['time_to_event'] = (
            pd.to_datetime(events_df['start_time']) - pd.to_datetime('now')
        ).dt.total_seconds() / 3600
        
        # Create feature matrix
        feature_matrix = np.zeros((len(self.event_id_to_idx), 3))  # time + sport + league
        
        for event_id, idx in self.event_id_to_idx.items():
            event = events_df[events_df['event_id'] == event_id].iloc[0]
            feature_matrix[idx] = [
                event['time_to_event'],
                hash(str(event['sport_id'])) % 100,  # Simple hash for categorical
                hash(str(event['league_id'])) % 100
            ]
        
        return feature_matrix

    def process_market_features(self, market_df):
        # Create feature matrix
        feature_matrix = np.zeros((len(self.market_id_to_idx), 1))  # odds only
        
        for market_id, idx in self.market_id_to_idx.items():
            market = market_df[market_df['market_id'] == market_id].iloc[0]
            feature_matrix[idx] = [market['avg_odds']]
        
        return feature_matrix

    def create_betslips(self, bets_df):
        betslips = []
        
        for bet_id, group in bets_df.groupby('bet_id'):
            n_selections = len(group)
            if n_selections > self.max_selections:
                continue
            
            try:
                betslip = {
                    'player_id': group['player_id'].iloc[0],
                    'event_ids': group['event_id'].tolist(),
                    'market_ids': group['market_id'].tolist(),
                    'odds': group['bet_odds'].tolist(),
                    'combined_odds': group['bet_odds'].prod(),
                    'n_selections': n_selections,
                    'label': 1.0 if group['status'].iloc[0] == 'won' else 0.0
                }
                betslips.append(betslip)
            except Exception as e:
                print(f"Error creating betslip for bet_id {bet_id}: {str(e)}")
                continue
        
        return betslips

    def __len__(self):
        return len(self.betslips)

    def __getitem__(self, idx):
        betslip = self.betslips[idx]
        
        try:
            # Get user features
            user_idx = self.user_id_to_idx[betslip['player_id']]
            user_features = self.user_features[user_idx]
            
            # Pad event and market features
            event_features = np.zeros((self.max_selections, self.event_features.shape[1]))
            market_features = np.zeros((self.max_selections, self.market_features.shape[1]))
            
            for i, (event_id, market_id) in enumerate(zip(betslip['event_ids'], betslip['market_ids'])):
                event_idx = self.event_id_to_idx[event_id]
                market_idx = self.market_id_to_idx[market_id]
                
                event_features[i] = self.event_features[event_idx]
                market_features[i] = self.market_features[market_idx]
            
            return {
                'user_features': torch.FloatTensor(user_features),
                'event_features': torch.FloatTensor(event_features),
                'market_features': torch.FloatTensor(market_features),
                'combined_odds': torch.FloatTensor([betslip['combined_odds']]),
                'n_selections': torch.LongTensor([betslip['n_selections']]),
                'label': torch.FloatTensor([betslip['label']])
            }
            
        except Exception as e:
            print(f"Error processing betslip {idx}: {str(e)}")
            # Return zero tensors as fallback
            return {
                'user_features': torch.zeros(self.user_features.shape[1]),
                'event_features': torch.zeros(self.max_selections, self.event_features.shape[1]),
                'market_features': torch.zeros(self.max_selections, self.market_features.shape[1]),
                'combined_odds': torch.zeros(1),
                'n_selections': torch.zeros(1, dtype=torch.long),
                'label': torch.zeros(1)
            }

In [216]:
class BetslipRecommenderModel(nn.Module):
    def __init__(self, user_dim, event_dim, market_dim, embedding_dim=64, max_selections=5):
        super().__init__()
        
        # Initialize weights with Xavier/Glorot initialization
        def init_weights(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0.1)
        
        self.max_selections = max_selections
        
        # User Tower
        self.user_tower = nn.Sequential(
            nn.Linear(user_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, embedding_dim)
        )
        self.user_tower.apply(init_weights)
        
        # Event Tower
        self.event_tower = nn.Sequential(
            nn.Linear(event_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, embedding_dim)
        )
        self.event_tower.apply(init_weights)
        
        # Market Tower
        self.market_tower = nn.Sequential(
            nn.Linear(market_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, embedding_dim)
        )
        self.market_tower.apply(init_weights)
        
        # Final prediction layers
        self.predictor = nn.Sequential(
            nn.Linear(embedding_dim + 2, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.predictor.apply(init_weights)

    def forward(self, user_features, event_features, market_features, combined_odds, n_selections):
        batch_size = user_features.size(0)
        
        # Add small noise to prevent overfitting
        if self.training:
            user_features = user_features + torch.randn_like(user_features) * 0.01
            event_features = event_features + torch.randn_like(event_features) * 0.01
            market_features = market_features + torch.randn_like(market_features) * 0.01
        
        # Process features
        user_emb = self.user_tower(user_features)
        
        # Process events and markets
        event_features_flat = event_features.view(-1, event_features.size(-1))
        market_features_flat = market_features.view(-1, market_features.size(-1))
        
        event_emb = self.event_tower(event_features_flat)
        market_emb = self.market_tower(market_features_flat)
        
        # Reshape back
        event_emb = event_emb.view(batch_size, self.max_selections, -1)
        market_emb = market_emb.view(batch_size, self.max_selections, -1)
        
        # Combine embeddings
        selection_emb = event_emb + market_emb
        
        # Average pooling over selections
        betslip_emb = torch.mean(selection_emb, dim=1)
        
        # Combine features
        combined_features = torch.cat([
            betslip_emb + user_emb,
            combined_odds,
            n_selections.float()
        ], dim=1)
        
        return self.predictor(combined_features)


In [217]:

def train_betslip_model(model, train_loader, val_loader, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Custom loss with regularization
    def custom_loss(outputs, labels, model):
        bce_loss = F.binary_cross_entropy_with_logits(outputs, labels)
        
        # L2 regularization
        l2_lambda = 0.01
        l2_reg = torch.tensor(0.).to(device)
        for param in model.parameters():
            l2_reg += torch.norm(param)
        
        return bce_loss + l2_lambda * l2_reg
    
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=0.001,
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )
    
    # Cosine annealing scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs,
        eta_min=1e-6
    )
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for batch_idx, batch in enumerate(train_loader):
            # Move data to device
            user_features = batch['user_features'].to(device)
            event_features = batch['event_features'].to(device)
            market_features = batch['market_features'].to(device)
            combined_odds = batch['combined_odds'].to(device)
            n_selections = batch['n_selections'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(
                user_features,
                event_features,
                market_features,
                combined_odds,
                n_selections
            )
            
            # Calculate loss
            loss = custom_loss(outputs, labels, model)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            # Calculate accuracy
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            train_correct += (predictions == labels).sum().item()
            train_total += labels.size(0)
            
            train_loss += loss.item()
            
            if batch_idx % 100 == 0:
                print(f'Batch {batch_idx}:')
                print(f'Loss: {loss.item():.4f}')
                print(f'Accuracy: {100 * train_correct/train_total:.2f}%')
                print(f'Learning Rate: {scheduler.get_last_lr()[0]:.6f}')
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch in val_loader:
                user_features = batch['user_features'].to(device)
                event_features = batch['event_features'].to(device)
                market_features = batch['market_features'].to(device)
                combined_odds = batch['combined_odds'].to(device)
                n_selections = batch['n_selections'].to(device)
                labels = batch['label'].to(device)
                
                outputs = model(
                    user_features,
                    event_features,
                    market_features,
                    combined_odds,
                    n_selections
                )
                
                loss = custom_loss(outputs, labels, model)
                val_loss += loss.item()
                
                predictions = (torch.sigmoid(outputs) > 0.5).float()
                val_correct += (predictions == labels).sum().item()
                val_total += labels.size(0)
        
        # Update learning rate
        scheduler.step()
        
        print(f"\nEpoch {epoch+1}/{num_epochs}:")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Train Accuracy: {100 * train_correct/train_total:.2f}%")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"Val Accuracy: {100 * val_correct/val_total:.2f}%")
        print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")
        print("-" * 50)


In [218]:
def generate_betslip_recommendations(
    model, user_id, available_events, market_df, dataset,
    top_k=5, max_selections=3, min_odds=1.5, max_combined_odds=10.0
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    
    print("Preparing to generate recommendations...")
    
    try:
        # Get user features
        user_idx = dataset.user_id_to_idx[user_id]
        user_features = dataset.user_features[user_idx]
        user_features = torch.FloatTensor(user_features).unsqueeze(0).to(device)
        
        candidate_betslips = []
        
        with torch.no_grad():
            # Single selections
            for event_id in available_events:
                if event_id not in dataset.event_id_to_idx:
                    continue
                    
                for market_id in dataset.market_id_to_idx.keys():
                    # Initialize feature arrays
                    event_features = np.zeros((dataset.max_selections, dataset.event_features.shape[1]))
                    market_features = np.zeros((dataset.max_selections, dataset.market_features.shape[1]))
                    
                    # Fill first selection
                    event_idx = dataset.event_id_to_idx[event_id]
                    market_idx = dataset.market_id_to_idx[market_id]
                    
                    event_features[0] = dataset.event_features[event_idx]
                    market_features[0] = dataset.market_features[market_idx]
                    
                    # Get odds
                    odds = float(market_df.loc[market_df['market_id'] == market_id, 'avg_odds'].iloc[0])
                    if odds < min_odds:
                        continue
                    
                    # Prepare model inputs
                    event_features = torch.FloatTensor(event_features).unsqueeze(0).to(device)
                    market_features = torch.FloatTensor(market_features).unsqueeze(0).to(device)
                    combined_odds = torch.FloatTensor([[odds]]).to(device)
                    n_selections = torch.LongTensor([[1]]).to(device)
                    
                    # Get prediction
                    score = model(
                        user_features,
                        event_features,
                        market_features,
                        combined_odds,
                        n_selections
                    )
                    
                    candidate_betslips.append({
                        'events': [event_id],
                        'markets': [market_id],
                        'odds': [odds],
                        'combined_odds': odds,
                        'score': score.item()
                    })
            
            # Multiple selections
            for n_sel in range(2, max_selections + 1):
                print(f"Generating {n_sel}-selection betslips...")
                
                for events in combinations(available_events, n_sel):
                    # Validate events
                    if not all(e in dataset.event_id_to_idx for e in events):
                        continue
                    
                    # Get markets for each event
                    markets_list = [dataset.market_id_to_idx.keys() for _ in range(n_sel)]
                    
                    for markets in product(*markets_list):
                        # Get odds
                        try:
                            odds = [
                                float(market_df.loc[market_df['market_id'] == m, 'odds'].iloc[0])
                                for m in markets
                            ]
                        except (IndexError, KeyError):
                            continue
                        
                        combined_odds = np.prod(odds)
                        if combined_odds > max_combined_odds or min(odds) < min_odds:
                            continue
                        
                        # Prepare features
                        event_features = np.zeros((dataset.max_selections, dataset.event_features.shape[1]))
                        market_features = np.zeros((dataset.max_selections, dataset.market_features.shape[1]))
                        
                        # Fill features
                        for i, (event_id, market_id) in enumerate(zip(events, markets)):
                            event_idx = dataset.event_id_to_idx[event_id]
                            market_idx = dataset.market_id_to_idx[market_id]
                            
                            event_features[i] = dataset.event_features[event_idx]
                            market_features[i] = dataset.market_features[market_idx]
                        
                        # Prepare model inputs
                        event_features = torch.FloatTensor(event_features).unsqueeze(0).to(device)
                        market_features = torch.FloatTensor(market_features).unsqueeze(0).to(device)
                        combined_odds_tensor = torch.FloatTensor([[combined_odds]]).to(device)
                        n_selections_tensor = torch.LongTensor([[n_sel]]).to(device)
                        
                        # Get prediction
                        score = model(
                            user_features,
                            event_features,
                            market_features,
                            combined_odds_tensor,
                            n_selections_tensor
                        )
                        
                        candidate_betslips.append({
                            'events': list(events),
                            'markets': list(markets),
                            'odds': odds,
                            'combined_odds': combined_odds,
                            'score': score.item()
                        })
        
        # Sort and return top-k
        sorted_betslips = sorted(candidate_betslips, key=lambda x: x['score'], reverse=True)
        return sorted_betslips[:top_k]
    
    except Exception as e:
        print(f"Error generating recommendations: {str(e)}")
        import traceback
        traceback.print_exc()
        return []

def get_betslip_details(betslip, event_df, market_df):
    events_info = []
    for event_id in betslip['events']:
        event = event_df[event_df['event_id'] == event_id].iloc[0]
        events_info.append(f"{event['home_team']} vs {event['away_team']}")
    
    markets_info = []
    for market_id in betslip['markets']:
        market = market_df[market_df['market_id'] == market_id].iloc[0]
        markets_info.append(f"{market['market_type']}")
    
    return events_info, markets_info


In [219]:
import pandas as pd
from recsys.config import Settings

settings = Settings()
SOURCE_DIR = settings.SOURCE_DATA_DIR
processed_dir = settings.PROCESSED_DATA_DIR
# Prepare data
user_df = pd.read_csv(SOURCE_DIR / 'users.csv')
event_df = pd.read_csv(SOURCE_DIR / 'events.csv')
bet_df = pd.read_csv(SOURCE_DIR / 'bets.csv')

In [220]:
bet_events = set(bet_df.event_id.to_list())
evnts = set(event_df.event_id.to_list())
list_of_events = evnts.union(bet_events)
event_id_converter = IDConverter()
for id in list_of_events:
    event_id_converter.convert(id)
player_id_converter = IDConverter()
user_df['player_id'] = user_df['player_id'].apply(player_id_converter.convert)
brand_id_converter = IDConverter()
user_df['brand_id'] = user_df['brand_id'].apply(brand_id_converter.convert)
# bet_id, outcome_id, market_id
bet_id_converter = IDConverter()
bet_df['bet_id'] = bet_df['bet_id'].apply(bet_id_converter.convert)
outcome_id_converter = IDConverter()
bet_df['outcome_id'] = bet_df['outcome_id'].apply(outcome_id_converter.convert)
market_id_converter = IDConverter()
bet_df['market_id'] = bet_df['market_id'].apply(market_id_converter.convert)

event_df['event_id'] = event_df['event_id'].map(event_id_converter.id_to_int)
sport_id_converter = IDConverter()
event_df['sport_id'] = event_df['sport_id'].apply(sport_id_converter.convert)
league_id_converter = IDConverter()
event_df['league_id'] = event_df['league_id'].apply(league_id_converter.convert)
bet_df.player_id = bet_df.player_id.map(player_id_converter.id_to_int)
bet_df.brand_id = bet_df.brand_id.map(brand_id_converter.id_to_int)
bet_df.event_id = bet_df.event_id.map(event_id_converter.id_to_int)
user_df = user_df.drop_duplicates().copy(deep=True)
event_df = event_df.drop_duplicates().copy(deep=True)
bet_df = bet_df.drop_duplicates().copy(deep= True)
user_df.player_reg_date = user_df.player_reg_date.str[:19]

In [221]:
bet_df['win_lose'] = bet_df.status.map({'win': 1, 'lose': 0})
bet_df.bet_date = pd.to_datetime(bet_df.bet_date.str[:19])

In [222]:

market_cols = ['market_id','win_lose', 'outcome_odds', 'bet_id']
market_df = bet_df[market_cols].copy()

In [223]:
market_df = market_df.groupby(['market_id']).agg({'bet_id': 'count', 'win_lose': 'mean', 'outcome_odds' : 'mean' }).reset_index()

In [224]:
market_df.columns = ['market_id', 'bet_count', 'win_rate', 'avg_odds' ]

In [225]:
# Prepare data
user_df = user_df.copy()
event_df = event_df.copy()
bets_df = bet_df.copy()
market_df = market_df.copy()


In [226]:

# Create dataset
dataset = BetslipDataset(user_df, event_df, bets_df, market_df)


Processed 200798 betslips
User features shape: (25621, 9)
Event features shape: (59387, 3)
Market features shape: (954, 3)


In [227]:

# Create data loaders
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [197]:

# Initialize model
def initialize_model(dataset):
    user_dim = dataset.user_features.shape[1]  # Number of user features
    event_dim = dataset.event_features.shape[1]  # Number of event features
    market_dim = dataset.market_features.shape[1]  # Number of market features
    
    print(f"User dimension: {user_dim}")
    print(f"Event dimension: {event_dim}")
    print(f"Market dimension: {market_dim}")
    
    model = BetslipRecommenderModel(
        user_dim=user_dim,
        event_dim=event_dim,
        market_dim=market_dim,
        embedding_dim=64,
        max_selections=dataset.max_selections
    )
    
    return model


In [198]:

# Check data distribution
def analyze_data(train_loader):
    positive_samples = 0
    total_samples = 0
    
    for batch in train_loader:
        labels = batch['label']
        positive_samples += labels.sum().item()
        total_samples += len(labels)
    
    print(f"Data Distribution:")
    print(f"Positive samples: {positive_samples}")
    print(f"Total samples: {total_samples}")
    print(f"Positive ratio: {100 * positive_samples/total_samples:.2f}%")

# Usage
print("Analyzing data distribution...")
analyze_data(train_loader)


Analyzing data distribution...
Data Distribution:
Positive samples: 0.0
Total samples: 36172
Positive ratio: 0.00%


In [ ]:

print("\nStarting training...")
model = initialize_model(dataset)
train_betslip_model(model, train_loader, val_loader)

In [ ]:

# Usage
model = initialize_model(dataset)
model = model.to(settings.DEVICE)

# Print model summary
print("\nModel Architecture:")
print(model)

# Check input shapes
sample_batch = next(iter(train_loader))
print("\nInput shapes:")
for k, v in sample_batch.items():
    print(f"{k}: {v.shape}")

In [191]:

# Train model
train_betslip_model(model, train_loader, val_loader)


/Users/sudhamshuaddanki/smaddanki/Assessments/Whizdomai/.venv/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Batch 0: Loss = 0.8838
NaN loss detected in batch 40
Batch 100: Loss = 0.1925
NaN loss detected in batch 127
NaN loss detected in batch 155
NaN loss detected in batch 166
NaN loss detected in batch 174
NaN loss detected in batch 188
NaN loss detected in batch 195
Batch 200: Loss = 0.1253
Batch 300: Loss = 0.0838
NaN loss detected in batch 314
NaN loss detected in batch 316
NaN loss detected in batch 324
NaN loss detected in batch 379
NaN loss detected in batch 399
Batch 400: Loss = 0.0635
NaN loss detected in batch 428
NaN loss detected in batch 442
Batch 500: Loss = 0.0538
NaN loss detected in batch 511
NaN loss detected in batch 524
NaN loss detected in batch 527
NaN loss detected in batch 543
NaN loss detected in batch 551
NaN loss detected in batch 573
NaN loss detected in batch 576
NaN loss detected in batch 581
Batch 600: Loss = 0.0454
NaN loss detected in batch 608
NaN loss detected in batch 615
NaN loss detected in batch 619
NaN loss detected in batch 637
NaN loss detected in b

BetslipRecommenderModel(
  (user_tower): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (event_tower): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (market_tower): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): LayerNorm((64,), eps=1e-05, eleme

In [187]:
# Usage
print("Generating recommendations...")
recommendations = generate_betslip_recommendations(
    model=model,
    user_id=101,  # Replace with actual user ID
    available_events=event_df['event_id'].tolist(),
    market_df=market_df,
    dataset=dataset
)

if recommendations:
    print("\nTop Recommendations:")
    for i, rec in enumerate(recommendations, 1):
        events_info, markets_info = get_betslip_details(rec, event_df, market_df)
        print(f"\nBetslip Recommendation {i}:")
        for j, (event, market, odd) in enumerate(zip(events_info, markets_info, rec['odds'])):
            print(f"Selection {j+1}:")
            print(f"  Event: {event}")
            print(f"  Market: {market}")
            print(f"  Odds: {odd:.2f}")
        print(f"Combined Odds: {rec['combined_odds']:.2f}")
        print(f"Confidence Score: {rec['score']:.4f}")
else:
    print("No recommendations generated.")

Generating recommendations...
Preparing to generate recommendations...


KeyboardInterrupt: 